In [1]:
import time
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.animation as anim
from mpl_toolkits.mplot3d import Axes3D
from sklearn.datasets import make_blobs

import sklearn.cluster as cluster


from parse_bin_output import *

# Kalman Filter
def KF(obs, state, P, A, H, Q, R):
    pred_state = A @ state
    pred_P = A @ P @ A.T + Q

    update_K = pred_P @ H.T @ np.linalg.inv(H @ pred_P @ H.T + R)
    update_state = pred_state + update_K @ (obs - H @ pred_state)
    update_P = (np.eye(len(update_state)) - update_K @ H) @ pred_P

    return update_state, update_P


def Count(pointCloud):
    # initialization
    cen_list = []

    state = np.zeros((4, 1))
    P = np.eye(4)
    dt = 1
    A = np.eye(4)
    A[0][2] = dt
    A[1][3] = dt
    H = np.eye(4)
    R = np.diag([0.1, 0.1, 0.1, 0.1])
    Q = np.diag([0.01, 0.01, 0.01, 0.01])

    for frame_num in range(len(pointCloud)):

        # same object
        if len(pointCloud[frame_num]) > min_pts:
            # cluster
            xyz = pointCloud[frame_num]
            # clf = Kmeans(k=1)
            # labels, centroids = clf.predict(xyz)

            dbscan = cluster.DBSCAN(eps=0.5, min_samples=min_pts)
            labels = dbscan.fit_predict(xyz)
            uni_labels = np.unique(labels)
            centroids = []

            for label in uni_labels:
                if label == -1: # noise
                    continue
                else:
                    points = []
                    for i in range(len(labels)):
                        if labels[i] == label:
                            points.append(xyz[i])
                    centroid = np.mean(points, axis=0)
                    centroids.append(centroid)
                    break # only want 1 centroid

            # Kalman Filter
            state, P = KF(np.concatenate([np.array(centroids).reshape(-1, 1), np.zeros((1, 1))]), state, P, A, H, Q, R)
            cen_list.append(state[:2].flatten())

        # new object
        if len(pointCloud[frame_num]) == 0 or (frame_num+1) % 25 == 0:
            # process trajectory
            if len(cen_list) < 3: # not enough data
                cen_list = []
                continue

            cen_list = np.array(cen_list)

            # plot trajectory
#             fig = plt.figure()
#             ax = fig.add_subplot(111, projection='3d')
#             ax.scatter(cen_list[:, 0], cen_list[:, 1], c='black', s=1, label = 'Object')
#             ax.set_xlabel('X')
#             ax.set_ylabel('Y')
#             ax.set_zlabel('Z')
#             ax.set_xlim(-1,1)
#             ax.set_ylim(0,3)
#             ax.set_zlim(-1,1)
#             ax.legend()
#             plt.show()

#             print(cen_list)

            global people_in, people_out, inside_room

            # Identify direction of movement in y-axis
            if cen_list[0][1] < cen_list[len(cen_list)-1][1]:
                people_in += 1
                inside_room += 1
            elif cen_list[0][1] > cen_list[len(cen_list)-1][1]:
                people_out += 1
                inside_room -= 1

            # refresh cen_list
            cen_list = []


people_in = 0
people_out = 0
inside_room = 0

min_pts = 5

# file path with real-time processing of last folder in directory (in this case it is set to demo)
#binFilePath = "../Pre-Lab 4/Industrial Visualizer/Industrial_Visualizer/binData/" + os.listdir("../Pre-Lab 4/Industrial Visualizer/Industrial_Visualizer/binData")[-1]
binFilePath = "demo"
time_end = time.time() + 60
read_len = 0

while True:
    if read_len < len(os.listdir(binFilePath)):
        output_dict = parse_file(binFilePath, read_len+1)

        pointCloud = []
        for frame in output_dict:
            if 'pointCloud' in frame:
                pointCloud.append(frame['pointCloud'][:,0:3])
        Count(pointCloud)

        time_end = time.time() + 60
        read_len += 1
        print("People in:", people_in)
        print("People out:", people_out)
        print("Inside the room:", inside_room)
        print('done processing file')
    if time.time() > time_end:
        break

print("People in:", people_in)
print("People out:", people_out)
print("Inside the room:", inside_room)

processing file: demo/pHistBytes_1.bin
People in: 3
People out: 1
Inside the room: 2
done processing file
